## Import libraries


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os 
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D 
from tensorflow.keras.layers import MaxPool2D, Flatten, Dense 
from tensorflow.keras import Model

## Extracting zip file

In [3]:
!pip install unzip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
!unzip /content/drive/MyDrive/dataset2/dataset.zip -d /content/sample_data/dataset/train

Archive:  /content/drive/MyDrive/dataset2/dataset.zip
replace /content/sample_data/dataset/train/X/melspec_1127.npy? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [5]:
!unzip /content/drive/MyDrive/dataset2/validation.zip -d /content/sample_data/dataset/val

Archive:  /content/drive/MyDrive/dataset2/validation.zip
replace /content/sample_data/dataset/val/dataset/X/melspec_1127.npy? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

## Loading dataset

In [6]:
TRAIN_ROOT = "/content/sample_data/dataset/train"
VAL_ROOT = "/content/sample_data/dataset/val/dataset"

In [7]:
train_x_file_names = sorted(os.listdir(os.path.join(TRAIN_ROOT,'X')))
train_y_file_names = sorted(os.listdir(os.path.join(TRAIN_ROOT,'Y')))
val_x_file_names = sorted(os.listdir(os.path.join(VAL_ROOT,'X')))
val_y_file_names = sorted(os.listdir(os.path.join(VAL_ROOT,'Y')))

In [8]:
X_train = []
y_train = []
X_val = []
y_val = []
for i in range(len(train_x_file_names)):
    spec = np.load(os.path.join(TRAIN_ROOT,'X',train_x_file_names[i]))
    spec = spec.reshape(spec.shape[1],spec.shape[2],spec.shape[0])
    X_train.append(spec)
    label = np.sum(np.load(os.path.join(TRAIN_ROOT,'Y',train_y_file_names[i])), axis=1)
    label = (label>0)
    y_train.append(label) 
for i in range(len(val_x_file_names)):
    spec = np.load(os.path.join(VAL_ROOT,'X',val_x_file_names[i]))
    spec = spec.reshape(spec.shape[1],spec.shape[2],spec.shape[0])
    X_val.append(spec)    
    label = np.sum(np.load(os.path.join(VAL_ROOT,'Y',val_y_file_names[i])), axis=1)
    label = (label>0)
    y_val.append(label) 

X_train = np.asarray(X_train)
y_train = np.asarray(y_train)
y_train = y_train.astype('float64')
X_val = np.asarray(X_val)
y_val = np.asarray(y_val)
y_val = y_val.astype('float64')

print(X_train.shape,y_train.shape,X_val.shape,y_val.shape)

(10000, 64, 1000, 1) (10000, 11) (2000, 64, 1000, 1) (2000, 11)


## Models

In [9]:
input = Input(shape =(64,1000,1))
# 1st Conv Block

x = Conv2D (filters =64, kernel_size =3, padding ='same', activation='relu')(input)
x = Conv2D (filters =64, kernel_size =3, padding ='same', activation='relu')(x)
x = MaxPool2D(pool_size =2, strides =2, padding ='same')(x)
# 2nd Conv Block

x = Conv2D (filters =128, kernel_size =3, padding ='same', activation='relu')(x)
x = Conv2D (filters =128, kernel_size =3, padding ='same', activation='relu')(x)
x = MaxPool2D(pool_size =2, strides =2, padding ='same')(x)
# 3rd Conv block

x = Conv2D (filters =256, kernel_size =3, padding ='same', activation='relu')(x)
x = Conv2D (filters =256, kernel_size =3, padding ='same', activation='relu')(x)
x = Conv2D (filters =256, kernel_size =3, padding ='same', activation='relu')(x)
x = MaxPool2D(pool_size =2, strides =2, padding ='same')(x)
# 4th Conv block

x = Conv2D (filters =512, kernel_size =3, padding ='same', activation='relu')(x)
x = Conv2D (filters =512, kernel_size =3, padding ='same', activation='relu')(x)
x = Conv2D (filters =512, kernel_size =3, padding ='same', activation='relu')(x)
x = MaxPool2D(pool_size =2, strides =2, padding ='same')(x)

# 5th Conv block

x = Conv2D (filters =512, kernel_size =3, padding ='same', activation='relu')(x)
x = Conv2D (filters =256, kernel_size =3, padding ='same', activation='relu')(x)
x = Conv2D (filters =128, kernel_size =3, padding ='same', activation='relu')(x)
x = MaxPool2D(pool_size =2, strides =2, padding ='same')(x)
# Fully connected layers

x = Flatten()(x)
x = Dense(units = 4096, activation ='relu')(x)
x = Dense(units = 512, activation ='relu')(x)
output = Dense(units = 11, activation ='sigmoid')(x)
# creating the model

model = Model (inputs=input, outputs =output)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 64, 1000, 1)]     0         
                                                                 
 conv2d (Conv2D)             (None, 64, 1000, 64)      640       
                                                                 
 conv2d_1 (Conv2D)           (None, 64, 1000, 64)      36928     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 32, 500, 64)      0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 32, 500, 128)      73856     
                                                                 
 conv2d_3 (Conv2D)           (None, 32, 500, 128)      147584    
                                                             

## Metric functions

In [10]:
from keras import backend as K
def recall(y_true, pred):
    TP =tf.math.reduce_sum((pred == y_true)& (y_true==1))
    TN =tf.math.reduce_sum((pred == y_true)& (y_true==0))
    FN =tf.math.reduce_sum((pred != y_true)& (y_true==1))
    FP =tf.math.reduce_sum((pred != y_true)& (y_true==0))
    recall=TP/(TP+FN)
    return recall

def precision(y_true, pred):
    TP =tf.math.reduce_sum((pred == y_true)& (y_true==1))
    TN =tf.math.reduce_sum((pred == y_true)& (y_true==0))
    FN =tf.math.reduce_sum((pred != y_true)& (y_true==1))
    FP =tf.math.reduce_sum((pred != y_true)& (y_true==0))
    precision=TP/(TP+FP)
    return precision

def f1(y_true, pred):
    TP =tf.math.reduce_sum((pred == y_true)& (y_true==1))
    TN =tf.math.reduce_sum((pred == y_true)& (y_true==0))
    FN =tf.math.reduce_sum((pred != y_true)& (y_true==1))
    FP =tf.math.reduce_sum((pred != y_true)& (y_true==0))
    precision=TP/(TP+FP)
    recall=TP/(TP+FN)
    f1=(2*precision*recall)/(precision+recall)
    return f1

## Training code

In [11]:
model.compile(loss='binary_crossentropy', optimizer='Adam', metrics=['accuracy'])

In [12]:
history = model.fit(X_train, y_train, validation_data= (X_val, y_val), batch_size=64, epochs=10, verbose=1)

Epoch 1/10
157/157 [==============================] - 213s 1s/step - loss: 0.4304 - accuracy: 0.0774 - val_loss: 0.4658 - val_accuracy: 0.0000e+00
Epoch 2/10
157/157 [==============================] - 186s 1s/step - loss: 0.4041 - accuracy: 0.0780 - val_loss: 0.4629 - val_accuracy: 0.0000e+00
Epoch 3/10
157/157 [==============================] - 186s 1s/step - loss: 0.4036 - accuracy: 0.0780 - val_loss: 0.4512 - val_accuracy: 0.0000e+00
Epoch 4/10
157/157 [==============================] - 192s 1s/step - loss: 0.4030 - accuracy: 0.0780 - val_loss: 0.4567 - val_accuracy: 0.0000e+00
Epoch 5/10
157/157 [==============================] - 183s 1s/step - loss: 0.4031 - accuracy: 0.0780 - val_loss: 0.4595 - val_accuracy: 0.0000e+00
Epoch 6/10
157/157 [==============================] - 183s 1s/step - loss: 0.4032 - accuracy: 0.0780 - val_loss: 0.4568 - val_accuracy: 0.0000e+00
Epoch 7/10
157/157 [==============================] - 183s 1s/step - loss: 0.4029 - accuracy: 0.0780 - val_loss: 0.458

In [ ]:
!unzip /content/drive/MyDrive/dataset2/test.zip -d /content/sample_data/dataset/test

In [14]:
TEST_ROOT = "/content/sample_data/dataset/test/test"

In [15]:
test_x_file_names = sorted(os.listdir(os.path.join(TEST_ROOT,'X')))
# train_y_file_names = sorted(os.listdir(os.path.join(TRAIN_ROOT,'Y')))